# 1.讀檔與資料處理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,auc,roc_curve
from sklearn import tree
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE 
import seaborn as sns

In [2]:
# pip install imblearn

In [3]:
# pip install yellowbrick

In [4]:
#ks.read_csv
#df.to_pandas()
Big_Table=pd.read_csv('./LA1_5lable.csv',index_col=[0],encoding='utf-8',low_memory=False)
Three_Day_Predict=pd.read_csv('./LA1_int.csv',index_col=[0],encoding='utf-8',low_memory=False)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
Big_Table.head()

,StationCode,StationNmae,ObservDay_time,StnPres,Temperature,RH,RHMin,WS,WD,WSGust,...,TradingVolume,T+1,T+3,T+5,T+7,T+1_label,T+3_label,T+5_label,T+7_label,month
locate,,,,,,,,,,,,,,,,,,,,,
Changhua,467270,田中,2020-04-09,1011.1,22.1,77.0,41.0,2.1,330.0,9.4,...,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0,4
Changhua,C0G620,芬園,2020-04-09,1003.2,22.0,75.0,32.0,1.4,7.0,8.8,...,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0,4
Changhua,C0G640,鹿港,2020-04-09,1017.3,21.2,73.0,54.0,2.4,8.0,9.8,...,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0,4
Changhua,C0G650,員林,2020-04-09,1012.2,22.5,79.0,37.0,0.8,332.0,7.2,...,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0,4
Changhua,C0G660,溪湖,2020-04-09,1014.8,22.1,71.0,41.0,2.5,4.0,9.5,...,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0,4


In [6]:
Big_Table.columns

Index(['StationCode', 'StationNmae', 'ObservDay_time', 'StnPres',
       'Temperature', 'RH', 'RHMin', 'WS', 'WD', 'WSGust', 'WDGust', 'Precp',
       'ty_generate', 'sea_warning', 'land_warning', 'ty_strength', 'ty_path',
       'festival_count', 'festival', 'altitude', 'altitude_classification',
       'locate_classification', 'Market', 'MediumAvgPrice', 'TradingVolume',
       'T+1', 'T+3', 'T+5', 'T+7', 'T+1_label', 'T+3_label', 'T+5_label',
       'T+7_label', 'month'],
      dtype='object')

In [51]:
Big_Table['T+3_label'].unique()

array([3., 4., 2., 5., 1.])

In [52]:
Big_Table['ObservDay_time']=pd.to_datetime(Big_Table['ObservDay_time'])

In [53]:
Big_Table['DayofWeek']=Big_Table['ObservDay_time'].dt.dayofweek

In [54]:
Big_Table_ordered=Big_Table[['StationCode', 'StationNmae', 'ObservDay_time', 'month', 'DayofWeek','StnPres',
       'Temperature', 'RH', 'RHMin', 'WS', 'WD', 'WSGust', 'WDGust', 'Precp',
       'ty_generate', 'sea_warning', 'land_warning', 'ty_strength', 'ty_path',
       'festival_count', 'festival', 'altitude', 'altitude_classification',
       'locate_classification', 'Market', 'MediumAvgPrice', 'TradingVolume',
       'T+1', 'T+3', 'T+5', 'T+7', 'T+1_label', 'T+3_label', 'T+5_label',
       'T+7_label', ]]
Big_Table_ordered

,StationCode,StationNmae,ObservDay_time,month,DayofWeek,StnPres,Temperature,RH,RHMin,WS,...,MediumAvgPrice,TradingVolume,T+1,T+3,T+5,T+7,T+1_label,T+3_label,T+5_label,T+7_label
locate,,,,,,,,,,,,,,,,,,,,,
Changhua,467270,田中,2020-04-09,4,3,1011.100000,22.100,77.000000,41.000000,2.10,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G620,芬園,2020-04-09,4,3,1003.200000,22.000,75.000000,32.000000,1.40,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G640,鹿港,2020-04-09,4,3,1017.300000,21.200,73.000000,54.000000,2.40,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G650,員林,2020-04-09,4,3,1012.200000,22.500,79.000000,37.000000,0.80,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
Changhua,C0G660,溪湖,2020-04-09,4,3,1014.800000,22.100,71.000000,41.000000,2.50,...,9.5,269804.0,0.021,0.053,0.079,0.168,3.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilan,C1U670,寒溪,2011-10-30,10,6,1004.350000,21.825,74.500000,66.500000,1.75,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0
Yilan,C1U690,新寮,2011-10-30,10,6,1004.350000,21.825,74.500000,66.500000,1.75,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0
Yunlin,C0K280,四湖,2011-10-30,10,6,1002.310135,23.500,72.868998,61.536329,3.20,...,13.1,135788.0,0.130,-0.233,-0.454,-0.260,4.0,2.0,1.0,1.0


In [11]:
#----------------------------我是分隔線------------------------------------------#

# 2. Correlation Analysis

In [55]:
###改drop T+3 成T+??
Big_Table_ordered_drop=Big_Table_ordered.drop(columns=['StationCode', 'StationNmae', 'ObservDay_time','RHMin','WSGust','festival','altitude', 'altitude_classification','locate_classification', 'Market', 'MediumAvgPrice',
       'T+1', 'T+3', 'T+5', 'T+7', 'T+1_label','T+5_label','T+7_label','TradingVolume','land_warning','ty_strength','ty_generate'])

In [56]:
Big_Table_ordered_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1356259 entries, Changhua to Yunlin
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   month           1356259 non-null  int64  
 1   DayofWeek       1356259 non-null  int64  
 2   StnPres         1356259 non-null  float64
 3   Temperature     1356259 non-null  float64
 4   RH              1356259 non-null  float64
 5   WS              1356259 non-null  float64
 6   WD              1356259 non-null  float64
 7   WDGust          1356259 non-null  float64
 8   Precp           1356259 non-null  float64
 9   sea_warning     1356259 non-null  int64  
 10  ty_path         1356259 non-null  int64  
 11  festival_count  1356259 non-null  int64  
 12  T+1_label       1356259 non-null  float64
dtypes: float64(8), int64(5)
memory usage: 177.1+ MB


In [57]:
Big_Table_ordered_drop.head(3)

,month,DayofWeek,StnPres,Temperature,RH,WS,WD,WDGust,Precp,sea_warning,ty_path,festival_count,T+1_label
locate,,,,,,,,,,,,,
Changhua,4,3,1011.1,22.1,77.0,2.1,330.0,340.0,0.0,0,0,0,3.0
Changhua,4,3,1003.2,22.0,75.0,1.4,7.0,341.0,0.0,0,0,0,3.0
Changhua,4,3,1017.3,21.2,73.0,2.4,8.0,7.0,0.0,0,0,0,3.0


In [18]:
#----------------------------我是分隔線------------------------------------------#

# 3. 建模：Random Forest Classifier

In [19]:
# 類樣本均衡審查
def label_samples_summary(Big_Table_ordered_drop):
    '''
    查看每個類的樣本量分布
    :param df: 數據框
    :return: 無
    '''
    print ('{:*^60}'.format('Labels samples count:'))
    print (Big_Table_ordered_drop.iloc[:, 1].groupby(Big_Table_ordered_drop.iloc[:, -1]).count()) # df.iloc[:,-1].value_counts()
    
label_samples_summary(Big_Table_ordered_drop)

*******************Labels samples count:********************
T+3_label
1.0    149830
2.0    392957
3.0    259070
4.0    348310
5.0    206092
Name: DayofWeek, dtype: int64


In [20]:
# 切分資料
X =Big_Table_ordered_drop.iloc[:,0:12]
y = Big_Table_ordered_drop.iloc[:,12]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)


print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (1085007, 12)
Number transactions y_train dataset:  (1085007,)
Number transactions X_test dataset:  (271252, 12)
Number transactions y_test dataset:  (271252,)


In [21]:
# 樣本均衡處理 (處理訓練樣品不平衡問題，丟到模型)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1.0)))
print("Before OverSampling, counts of label '2': {} \n".format(sum(y_train==2.0)))
print("Before OverSampling, counts of label '3': {} \n".format(sum(y_train==3.0)))
print("Before OverSampling, counts of label '4': {} \n".format(sum(y_train==4.0)))
print("Before OverSampling, counts of label '5': {} \n".format(sum(y_train==5.0)))


sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1.0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_train_res==2.0)))
print("After OverSampling, counts of label '3': {}".format(sum(y_train_res==3.0)))
print("After OverSampling, counts of label '4': {}".format(sum(y_train_res==4.0)))
print("After OverSampling, counts of label '5': {}".format(sum(y_train_res==5.0)))


Before OverSampling, counts of label '1': 119868
Before OverSampling, counts of label '2': 314603 

Before OverSampling, counts of label '3': 206975 

Before OverSampling, counts of label '4': 278609 

Before OverSampling, counts of label '5': 164952 

After OverSampling, the shape of train_X: (1573015, 12)
After OverSampling, the shape of train_y: (1573015,) 

After OverSampling, counts of label '1': 314603
After OverSampling, counts of label '2': 314603
After OverSampling, counts of label '3': 314603
After OverSampling, counts of label '4': 314603
After OverSampling, counts of label '5': 314603


In [22]:
# #搜尋最佳參數（會跑很久）
# from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
# #create a new rf classifier
# rf = RandomForestClassifier()

# #create a dictionary of all values we want to test for n_estimators
# params_rf = {'bootstrap': [True, False],
#              'max_depth': [10, 30, 50, None],
#              'max_features': ['auto', 'sqrt'],
#              'min_samples_leaf': [1, 2, 4],
#              'min_samples_split': [2, 5, 10],
#              'n_estimators': [50, 100]}

# #use gridsearch to test all values for n_estimators
# rf_gs = GridSearchCV(rf, params_rf,class_weight='balanced', cv=5,scoring="roc_auc",n_jobs=-1)
# #rf_gs = RandomizedSearchCV(rf, params_rf, scoring='roc_auc', cv=5, n_jobs=-1)

# #fit model to training data
model = RandomForestClassifier(max_depth=25, n_estimators=300,class_weight='balanced') #參數調整擺這裡

model.fit(X_train_res, y_train_res.ravel())

# rf_gs.fit(X_train, y_train) 
# #參數調整擺這裡
# #,class_weight='balanced
          
# #save best model
# rf_best = rf_gs.best_estimator_

# #check best n_estimators value
# print(rf_gs.best_params_)

# #可測參數：
# #{'bootstrap': [True, False],
# # 'max_depth': [10, 60, 110, None],
# # 'max_features': ['auto', 'sqrt'],
# # 'min_samples_leaf': [1, 2, 4],
# # 'min_samples_split': [2, 5, 10],
# # 'n_estimators': [200, 300, 400],'class_weight':["balanced"]
# #}

RandomForestClassifier(class_weight='balanced', max_depth=25, n_estimators=300,
                       n_jobs=-1)

In [23]:
# #原始資料評分
print('RandomForest_score_train : {}'.format(model.score(X_train_res,y_train_res)))
print('RandomForest_score_test : {}'.format(model.score(X_test, y_test)))

RandomForest_score_train : 0.9528847468078817
RandomForest_score_test : 0.5922905637562119


In [24]:
###改10

#Normalization :random forest不需要
# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)


# Make predictions using the testing set
y_pred =model.predict(X_test)

con_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [25]:
y_all=model.predict(X)

In [26]:
y_all

array([2., 2., 4., ..., 2., 2., 2.])

In [29]:
Big_Table_ordered_drop['T+3_y_predict'] = y_all.tolist()

Big_Table_ordered_drop1 = Big_Table_ordered_drop.assign(ObservDay_time=Big_Table_ordered['ObservDay_time'])
# Big_Table_ordered_drop.append(Big_Table_ordered['ObservDay_time'],ind)???

In [31]:
Big_Table_ordered_drop1

,month,DayofWeek,StnPres,Temperature,RH,WS,WD,WDGust,Precp,sea_warning,ty_path,festival_count,T+3_label,T+3_y_predict,ObservDay_time
locate,,,,,,,,,,,,,,,
Changhua,4,3,1011.100000,22.100,77.000000,2.10,330.000,340.000000,0.0,0,0,0,4.0,2.0,2020-04-09
Changhua,4,3,1003.200000,22.000,75.000000,1.40,7.000,341.000000,0.0,0,0,0,4.0,2.0,2020-04-09
Changhua,4,3,1017.300000,21.200,73.000000,2.40,8.000,7.000000,0.0,0,0,0,4.0,4.0,2020-04-09
Changhua,4,3,1012.200000,22.500,79.000000,0.80,332.000,351.000000,0.0,0,0,0,4.0,2.0,2020-04-09
Changhua,4,3,1014.800000,22.100,71.000000,2.50,4.000,332.000000,0.0,0,0,0,4.0,4.0,2020-04-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilan,10,6,1004.350000,21.825,74.500000,1.75,169.625,115.375000,6.0,0,0,0,2.0,2.0,2011-10-30
Yilan,10,6,1004.350000,21.825,74.500000,1.75,169.625,115.375000,13.0,0,0,0,2.0,2.0,2011-10-30
Yunlin,10,6,1002.310135,23.500,72.868998,3.20,20.000,132.082749,0.0,0,0,0,2.0,2.0,2011-10-30


In [50]:
a=Big_Table_ordered_drop1.groupby(['ObservDay_time'])
b=a.mean().astype(int)
b.head(100)

Final_T3_predict=b[['T+3_label','T+3_y_predict']]
Final_T3_predict.head(100)
# Final_T3_predict.to_csv('/Users/oliviah/Documents/Documents-2021-02-27/BDSE/Final_Project/Modeling_test/Final_T3_predict.csv')

,T+3_label,T+3_y_predict
ObservDay_time,,
2000-01-02,2,2
2000-01-03,4,3
2000-01-04,5,4
2000-01-05,4,3
2000-01-06,1,1
...,...,...
2000-04-26,5,4
2000-04-27,2,3
2000-04-28,3,3


In [ ]:

target_names = ['1', '2', '3','4','5']
print(classification_report(y_test, y_pred, target_names=target_names))
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
sns.heatmap(con_matrix.T, square=True, annot=True, fmt='d', cbar=False,cmap="YlGnBu",xticklabels=['1','2','3','4','5'], yticklabels=['1','2','3','4','5'])
plt.xlabel('true label',)
plt.ylabel('predicted label');
plt.title('Confusion Matrix')

In [ ]:
###改8.10

names=X.columns
imp_rf=model.feature_importances_
#把欄位名稱跟特徵分數結合(排序)
zip(imp_rf,names)
imp_rf,names=zip(*sorted(zip(imp_rf,names)))

#畫圖
plt.figure(figsize=(12,8))
plt.barh(names,imp_rf)
plt.xlabel('Importance of Feature')
plt.ylabel('Features')
plt.show()

# 4. Plot ROC curves:看模型好壞

In [ ]:
# # yellowbrick.classifier.base.ClassificationScoreVisualizer
# from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
# from yellowbrick.classifier import ROCAUC


# # Load multi-class classification dataset

# # Encode the non-numeric columns
# X = OrdinalEncoder().fit_transform(X)
# y = LabelEncoder().fit_transform(y)

# # Create the train and test data
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# # Instaniate the classification model and visualizer

# visualizer = ROCAUC(model, classes=["1", "2", "3","4","5","6","7"])

# visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
# visualizer.score(X_test, y_test)        # Evaluate the model on the test data
# visualizer.show()                       # Finalize and render the figure

In [ ]:
#----------------------------我是分隔線------------------------------------------#

# 5. Learning Curve  

In [ ]:
#learning curve測試（會跑很久）

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve


# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(model, 
                                                        X=X, 
                                                        y=y,
                                                        # Number of folds in cross-validation
                                                        cv=5,
                                                       
                                                        # Evaluation metric
                                                        scoring='f1_weighted',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 0.5, 5))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()